<a href="https://colab.research.google.com/github/evsemenova/Python-for-Analysts/blob/HW_5/%D0%A3%D1%80%D0%BE%D0%BA_5_%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0_%D0%B2_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B5_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%D0%B0_%D1%81_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%BE%D0%B9_Scipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Новый раздел

**1. Взять датасет из домашнего задания №2. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.**

In [26]:
#Импортируем нужные библиотеки
import pandas as pd
import scipy.stats as stats

In [27]:
df_marketing = pd.read_csv('marketing_campaign.csv')

In [28]:
df_subscribers = pd.read_csv('subscribers.csv')

In [29]:
df_users = pd.read_csv('users.csv')

In [30]:
df_marketing.head(1)

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True


In [31]:
df_subscribers.head(1)

,user_id,subscribing_channel,date_subscribed,date_canceled,is_retained
0,a1000,House Ads,1/1/18,NaN,True


In [32]:
df_users.head(1)

,user_id,age_group,language_preferred
0,a1000,0-18 years,English


In [33]:
# Объеденим таблицы
df_new = df_marketing.merge(df_subscribers, on='user_id', how='inner').merge(df_users, on='user_id', how='inner')

In [34]:
df_new.head(1)

,user_id,date_served,marketing_channel,variant,language_displayed,converted,subscribing_channel,date_subscribed,date_canceled,is_retained,age_group,language_preferred
0,a1000,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,0-18 years,English


In [35]:
# Создадим таблицу сопряженности
df_conjugacy = df.groupby('variant')['converted'].value_counts().unstack()
df_conjugacy.head()




converted,False,True
variant,,
control,4705,371
personalization,4241,705


In [36]:
# определяем значимость аб-теста
result = stats.chi2_contingency(df_conjugacy, correction = False)
chisq, pvalue = result[:2]
print('chisq = {}, pvalue = {}'.format(chisq, pvalue))

chisq = 126.07767827548582, pvalue = 2.9568009705199523e-29


В personalization конверсия выше.